# Day 06: Deployment & Monitoring

## Week 24 - Capstone Project

### Learning Objectives
- Design production deployment architecture
- Implement model versioning and registry
- Build monitoring dashboards for trading systems
- Create alerting systems for model drift and performance
- Set up automated retraining pipelines

### Why Deployment Matters
A model in a notebook is worthless. Production deployment transforms research into **real trading profits**.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json
import hashlib
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, field, asdict
from enum import Enum
from abc import ABC, abstractmethod
import pickle
from pathlib import Path

np.random.seed(42)
print("✅ Libraries loaded!")

## 1. Model Registry System

In [ ]:
class ModelStage(Enum):
    DEVELOPMENT = "development"
    STAGING = "staging"
    PRODUCTION = "production"
    ARCHIVED = "archived"


@dataclass
class ModelMetadata:
    """Metadata for a registered model."""
    model_id: str
    name: str
    version: str
    stage: ModelStage
    created_at: datetime
    metrics: Dict[str, float]
    parameters: Dict[str, Any]
    description: str
    tags: List[str] = field(default_factory=list)
    
    def to_dict(self) -> dict:
        return {
            'model_id': self.model_id,
            'name': self.name,
            'version': self.version,
            'stage': self.stage.value,
            'created_at': self.created_at.isoformat(),
            'metrics': self.metrics,
            'parameters': self.parameters,
            'description': self.description,
            'tags': self.tags
        }


class ModelRegistry:
    """Simple model registry for versioning and tracking."""
    
    def __init__(self, registry_path: str = './model_registry'):
        self.registry_path = Path(registry_path)
        self.registry_path.mkdir(parents=True, exist_ok=True)
        self.models: Dict[str, ModelMetadata] = {}
        self._load_registry()
    
    def _generate_id(self, name: str, version: str) -> str:
        content = f"{name}_{version}_{datetime.now().isoformat()}"
        return hashlib.md5(content.encode()).hexdigest()[:12]
    
    def register(self, name: str, version: str, model: Any,
                 metrics: Dict[str, float], parameters: Dict[str, Any],
                 description: str = "", tags: List[str] = None) -> str:
        """Register a new model."""
        model_id = self._generate_id(name, version)
        
        metadata = ModelMetadata(
            model_id=model_id,
            name=name,
            version=version,
            stage=ModelStage.DEVELOPMENT,
            created_at=datetime.now(),
            metrics=metrics,
            parameters=parameters,
            description=description,
            tags=tags or []
        )
        
        # Save model artifact
        model_path = self.registry_path / f"{model_id}.pkl"
        with open(model_path, 'wb') as f:
            pickle.dump(model, f)
        
        self.models[model_id] = metadata
        self._save_registry()
        
        print(f"✅ Model registered: {name} v{version} (ID: {model_id})")
        return model_id
    
    def promote(self, model_id: str, stage: ModelStage):
        """Promote model to a new stage."""
        if model_id not in self.models:
            raise ValueError(f"Model {model_id} not found")
        
        old_stage = self.models[model_id].stage
        self.models[model_id].stage = stage
        self._save_registry()
        
        print(f"✅ Model {model_id}: {old_stage.value} → {stage.value}")
    
    def get_production_model(self, name: str) -> Optional[Any]:
        """Get the production model for a given name."""
        for model_id, metadata in self.models.items():
            if metadata.name == name and metadata.stage == ModelStage.PRODUCTION:
                model_path = self.registry_path / f"{model_id}.pkl"
                with open(model_path, 'rb') as f:
                    return pickle.load(f)
        return None
    
    def list_models(self) -> pd.DataFrame:
        """List all registered models."""
        records = [m.to_dict() for m in self.models.values()]
        return pd.DataFrame(records)
    
    def _save_registry(self):
        registry_file = self.registry_path / 'registry.json'
        data = {k: v.to_dict() for k, v in self.models.items()}
        with open(registry_file, 'w') as f:
            json.dump(data, f, indent=2)
    
    def _load_registry(self):
        registry_file = self.registry_path / 'registry.json'
        if registry_file.exists():
            with open(registry_file, 'r') as f:
                data = json.load(f)
            # Reconstruct ModelMetadata objects
            for model_id, meta in data.items():
                self.models[model_id] = ModelMetadata(
                    model_id=meta['model_id'],
                    name=meta['name'],
                    version=meta['version'],
                    stage=ModelStage(meta['stage']),
                    created_at=datetime.fromisoformat(meta['created_at']),
                    metrics=meta['metrics'],
                    parameters=meta['parameters'],
                    description=meta['description'],
                    tags=meta.get('tags', [])
                )

print("✅ Model Registry implemented!")

## 2. Monitoring System

In [ ]:
@dataclass
class Alert:
    """Trading system alert."""
    timestamp: datetime
    severity: str  # 'INFO', 'WARNING', 'CRITICAL'
    category: str  # 'PERFORMANCE', 'DRIFT', 'RISK', 'SYSTEM'
    message: str
    details: Dict[str, Any] = field(default_factory=dict)


class TradingMonitor:
    """Monitor trading system health and performance."""
    
    def __init__(self):
        self.alerts: List[Alert] = []
        self.metrics_history: List[Dict] = []
        
        # Thresholds
        self.sharpe_warning = 0.5
        self.sharpe_critical = 0.0
        self.drawdown_warning = 0.10
        self.drawdown_critical = 0.20
        self.win_rate_warning = 0.45
    
    def check_performance(self, returns: pd.Series, window: int = 30) -> List[Alert]:
        """Check recent performance metrics."""
        recent = returns.tail(window)
        new_alerts = []
        
        # Sharpe ratio
        sharpe = recent.mean() / recent.std() * np.sqrt(252) if recent.std() > 0 else 0
        
        if sharpe < self.sharpe_critical:
            new_alerts.append(Alert(
                timestamp=datetime.now(),
                severity='CRITICAL',
                category='PERFORMANCE',
                message=f'Sharpe ratio critically low: {sharpe:.2f}',
                details={'sharpe': sharpe, 'window': window}
            ))
        elif sharpe < self.sharpe_warning:
            new_alerts.append(Alert(
                timestamp=datetime.now(),
                severity='WARNING',
                category='PERFORMANCE',
                message=f'Sharpe ratio below target: {sharpe:.2f}',
                details={'sharpe': sharpe, 'window': window}
            ))
        
        # Drawdown
        cumulative = (1 + recent).cumprod()
        drawdown = (cumulative / cumulative.cummax() - 1).min()
        
        if abs(drawdown) > self.drawdown_critical:
            new_alerts.append(Alert(
                timestamp=datetime.now(),
                severity='CRITICAL',
                category='RISK',
                message=f'Critical drawdown: {drawdown*100:.1f}%',
                details={'drawdown': drawdown, 'window': window}
            ))
        elif abs(drawdown) > self.drawdown_warning:
            new_alerts.append(Alert(
                timestamp=datetime.now(),
                severity='WARNING',
                category='RISK',
                message=f'Elevated drawdown: {drawdown*100:.1f}%',
                details={'drawdown': drawdown, 'window': window}
            ))
        
        self.alerts.extend(new_alerts)
        return new_alerts
    
    def check_model_drift(self, predictions: np.ndarray, actuals: np.ndarray,
                          baseline_accuracy: float = 0.55) -> List[Alert]:
        """Check for model drift by comparing accuracy."""
        new_alerts = []
        
        current_accuracy = np.mean(predictions == actuals)
        drift = baseline_accuracy - current_accuracy
        
        if drift > 0.10:
            new_alerts.append(Alert(
                timestamp=datetime.now(),
                severity='CRITICAL',
                category='DRIFT',
                message=f'Significant model drift detected: {drift*100:.1f}% drop in accuracy',
                details={'current_accuracy': current_accuracy, 'baseline': baseline_accuracy}
            ))
        elif drift > 0.05:
            new_alerts.append(Alert(
                timestamp=datetime.now(),
                severity='WARNING',
                category='DRIFT',
                message=f'Model drift detected: {drift*100:.1f}% drop in accuracy',
                details={'current_accuracy': current_accuracy, 'baseline': baseline_accuracy}
            ))
        
        self.alerts.extend(new_alerts)
        return new_alerts
    
    def generate_report(self) -> str:
        """Generate monitoring report."""
        report = f"""
{'='*60}
         TRADING SYSTEM MONITORING REPORT
{'='*60}
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

📊 ALERT SUMMARY
{'-'*40}
Critical: {len([a for a in self.alerts if a.severity == 'CRITICAL'])}
Warning:  {len([a for a in self.alerts if a.severity == 'WARNING'])}
Info:     {len([a for a in self.alerts if a.severity == 'INFO'])}

🚨 RECENT ALERTS
{'-'*40}
"""
        for alert in self.alerts[-10:]:
            icon = '🔴' if alert.severity == 'CRITICAL' else '🟡' if alert.severity == 'WARNING' else '🔵'
            report += f"{icon} [{alert.severity}] {alert.category}: {alert.message}\n"
        
        return report

print("✅ Monitoring system implemented!")

## 3. Example Deployment

In [ ]:
# Download data for demonstration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']

print("📥 Loading data for deployment demo...")
data = yf.download(TICKERS, start='2023-01-01', progress=False, auto_adjust=True)
prices = data['Close'].dropna()
returns = prices.pct_change().dropna()

# Create simple model (for demo)
class SimpleMomentumModel:
    def __init__(self, lookback: int = 20):
        self.lookback = lookback
    
    def predict(self, prices: pd.DataFrame) -> pd.DataFrame:
        momentum = prices / prices.shift(self.lookback) - 1
        signals = np.where(momentum > 0.02, 1, np.where(momentum < -0.02, -1, 0))
        return pd.DataFrame(signals, index=prices.index, columns=prices.columns)

# Register model
registry = ModelRegistry('./demo_registry')

model = SimpleMomentumModel(lookback=20)
model_id = registry.register(
    name='MomentumStrategy',
    version='1.0.0',
    model=model,
    metrics={'backtest_sharpe': 1.2, 'win_rate': 0.54},
    parameters={'lookback': 20},
    description='Simple momentum strategy for 5 US stocks',
    tags=['momentum', 'equity', 'us']
)

# Promote to production
registry.promote(model_id, ModelStage.PRODUCTION)

# List models
print("\n📋 Registered Models:")
print(registry.list_models()[['name', 'version', 'stage', 'created_at']])

In [ ]:
# Monitor system
monitor = TradingMonitor()

# Check performance
portfolio_returns = returns.mean(axis=1)  # Equal weight
alerts = monitor.check_performance(portfolio_returns, window=30)

print("\n" + monitor.generate_report())

## 4. Today's Trading Signals

In [ ]:
# Get production model
prod_model = registry.get_production_model('MomentumStrategy')

# Generate signals
signals = prod_model.predict(prices)
latest_signals = signals.iloc[-1]

print("\n" + "="*70)
print("🎯 PRODUCTION MODEL SIGNALS")
print("="*70)
print(f"Model: MomentumStrategy v1.0.0")
print(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
print(f"Status: PRODUCTION\n")

for ticker in TICKERS:
    signal = latest_signals[ticker]
    price = prices[ticker].iloc[-1]
    
    if signal == 1:
        signal_str = "🟢 BUY"
        action = "CALL options or shares"
    elif signal == -1:
        signal_str = "🔴 SELL"
        action = "PUT options or short"
    else:
        signal_str = "⚪ HOLD"
        action = "No action"
    
    print(f"{ticker:>6}: ${price:>8.2f}  {signal_str}  → {action}")

print("\n" + "="*70)
print("⚠️ Disclaimer: For educational purposes only.")
print("="*70)

## 5. Key Takeaways

### Deployment Components:
1. **Model Registry**: Version control for models
2. **Stage Management**: Dev → Staging → Production
3. **Monitoring**: Performance and drift detection
4. **Alerting**: Automated notifications

### Tomorrow: Presentation & Documentation
- Final project presentation
- Documentation standards
- Portfolio preparation